In [1]:
from transformers import (AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,TrainingArguments,Trainer)
from peft import LoraConfig,get_peft_model
import torch
from datasets import load_dataset

print(torch.cuda.is_available())

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



True


In [2]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:2000]")

def format_fn(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return {"text": prompt}

dataset = dataset.map(format_fn)

In [3]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_compute_dtype=torch.float16,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4"

)

In [8]:
model_name = 'mistralai/Mistral-7B-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  torch_dtype = torch.float16,
  device_map = {"" : 0}
)

lora_config = LoraConfig(
  r=8,
  lora_alpha=16,
  target_modules= ["q_proj","v_proj"],
  lora_dropout= 0.05,
  bias='none',
  task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.91s/it]


trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [9]:
def tokenize_fn(ex):
    output = tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    output["labels"] = output["input_ids"].copy()
    return output

tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=dataset.column_names)

In [10]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    max_steps=20,             # <--- limits run to 20 steps (test)
    fp16=True,
    logging_steps=5,
    output_dir="./mistral-lora-test",
    save_strategy="no",
    report_to="none",
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("\n Starting 20-step test training run...\n")
trainer.train()

print("\n Test LoRA run complete! Model + adapters in './mistral-lora-test'\n")
model.save_pretrained("./mistral-lora-test")


🚀 Starting 20-step test training run...



Step,Training Loss
5,5.480500
10,5.297000
15,5.270700
20,5.244900



✅ Test LoRA run complete! Model + adapters in './mistral-lora-test'

